In [1]:
import sys

sys.path.insert(0, "..") # dalgebra is here
from dalgebra import *
from dalgebra.hierarchies import *
from sage.categories.pushout import pushout
%display latex

# **Non-trivial centralizers and the spectral curve** $\def\ord{\text{ord}}\def\nn{\mathbb{N}}$

In this notebook we are going to explain and provide code on how we can get some non-trivial centralizers in differential algebra and, once a couple of commuting operators are found, we will compute the spectral curve for both.

This notebook will serve as a checking on the state on the repository and will push also the theory we have been developing further.

*Notation*: we will use the notation $(R,\partial)$ to described a differential ring and $R\{u\}$ as the ring of differential polynomials in $u$ (meaning, $R\{u\} = R[u,u',u'',\ldots]$ and all the derivations of $u$ are algebraically independent). A differential operator with coefficients over a differential ring $(R,\partial)$ is an element of the first Weyl algebra $R[\partial]$. This is a non-commutative ring where $\partial r = r' + r\partial$.

#### References

This work is based on the following references (will increase when needed):

1. *Computing Almost-commuting Basis of Ordinary Differential Operators*, A. Jiménez-Pastor, S. Rueda, M.A. Zurro, in the 48th International Symposium on Symbolic and Algebraic Computation (**short communication**), 2023. [(Extended notebook version)](https://github.com/Antonio-JP/dalgebra/blob/ISSAC-2023/notebooks/ISSAC23_Poster.ipynb)
2. *A *REDUCE* program for the integration of diferential polynomials*, A.H. Bilge in **Computer Physics Communications** (73-3), pp. 263-268, 1992. [doi:10.1016/0010-4655(92)90013-o](https://doi.org/10.1016/0010-4655(92)90013-o)
3. *Towards the classification of scalar nonpolynomial evolution equations: Quasilinearity*, A.H. Bilge in **Computers and Mathematics with Applications** (49-11,12), pp. 1837-1848, 2005. [doi:10.1016/j.camwa.2004.09.014](https://doi.org/10.1016/j.camwa.2004.09.014)
4. *Additive normal forms and integration of differential fractions*, F. Boulier, F. Lemaire, J. Lallemand, G. Regensburger, M. Rosenkranz, in **Journal of Symbolic Computation** (77), pp. 16-38, 2016. [doi:10.1016/j.jsc.2016.01.002](https://doi.org/10.1016/j.jsc.2016.01.002)
5. *Factorization of KdV Schrödinger operators using differential subresultants*, J.J. Morales-Ruíz, S. Rueda, M.A. Zurro in **Advances in Applied Mathematics** (120), pp. 102065, 2020. [doi:10.1016/j.aam.2020.102065](https://doi.org/10.1016/j.aam.2020.102065)
6. *Factoring Third Order Ordinary Differential Operators over Spectral Curves*, S. Rueda, M.A. Zurro in **ArXiV** [arXiv:2102.04733v1](https://www.arxiv.org/abs/2102.04733v1)
7. *Algebraic curves and soliton equations*, G. Wilson in **Geometry Today**, pp. 303-329, 1985.

## 1. Conditions for commutation

Following the work on [1], given a generic linear differential operator $L$ of order $n$ in normal form (i.e., without a term of order $n-1$) there is a set of almost-commuting differential operators $Q(L) \subset C\{u\}[\partial]$ that form a $C$-vector space. An operator $P \in C\{u\}[\partial]$ is said to *almost-commute* with $L$ if $\ord([L,P]) \leq n-2$. It is interesting to remark that this is not a symmetric property, meaning that if $P$ almost-commute with $L$, it does not imply $L$ almost-commute with $P$.

Let $L = \partial^n + u_{n-2}\partial^{n-2} + \ldots u_1\partial + u_0$ be defined in $C\{u_0,\ldots,u_{n-2}\} = C\{u\}$.

It was shown in by Wilson [7] that the set $Q(L)$ enjoys a basis of the form $\{P_m(u)\}_{m\in \nn}$, where
* $P_m(u) = \partial^m + p_{m,2}(u)\partial^{m-2} + \ldots + p_{m,m-1}(u)\partial + p_{m,m}(u)$ for some $p_{m,i}(u) \in C\{u\}$.
* If $u_i^{(k)}$ is set to have weight $n - i + k$, then $p_{m,j}(u)$ is homogeneous of weight $j$.

Wilson also proved that this basis is unique and that $P_m$ could be computed using $(L^{m/n})_+$, where $L^{1/n} \in C\{u\}((\partial^{-1}))$ and $(\cdot)_+$ just keep the differential part of pseudodifferential operators in $C\{u\}((\partial^{-1}))$. The computation of $P_m(u)$ using Wilson's formula was not efficient for higher values of $n$. We provided in [1] a simpler ansatz approach based only in linear algebra to compute the elements $P_m(u)$ of the basis.

This computation of $P_m(u)$ can also be done using integration of differential polynomials in several differential variables. This would relate with [2] and [4]. See [Section 1.1](#1.1.-The-differential-system-of-almost-commuting).

The computation of $P_m(u)$ provides two output simultaneously:
* The actual element $P_m(u)$ described by Wilson. This can be used to generate candidates for computing commutators once we have fixed a specific value for $L$.
* A set of $(n-1)$ non-linear differential equations in $u = (u_0, \ldots, u_{n-2})$. If the values for $u$ are given such that all equations vanish, then the given $\tilde{P}_m = P_m(\tilde{u})$ will commute with $\tilde{L} = L(\tilde{u})$. We denote these equations with $H_i^{(n,m)}$ with $i = 0,\ldots,n-2$. Namely:
$$[L,P_m] = \sum_{j=0}^{n-2} H_j^{(n,m)}(u) \partial^j.$$
  (We omit $(n,m)$ when it is clear from context, writing simply $H_i(u)$).

This is implemented in `dalgebra.hierarchies`:

In [2]:
P_4, eqs = almost_commuting_schr(3, 4)

In [3]:
show(P_4.parent()) # `z` is the representation of \partial
P_4

Ring of operator polynomials in (u0, u1, z) over Differential Ring [[Rational Field], (0,)]

2/9*u1_0^2*z_0 + 2/3*u0_1*z_0 + 4/3*u0_0*z_1 + 2/9*u1_2*z_0 + 2/3*u1_1*z_1 + 4/3*u1_0*z_2 + z_4

In [4]:
# Building the corresponding L or order 3
u0,u1,z = P_4.parent().gens()
L = z[3] + u1*z[1] + u0; L

u1_0*z_1 + u0_0 + z_3

In [5]:
# equations on u1 and u0 for P_4 to commute with L
show(eqs[0]) # H_0^(3,4)
show(eqs[1]) # H_1^(3,4)

4/9*u1_1*u1_0^2 + (-2/3)*u0_2*u1_0 + (-4/3)*u0_1*u0_0 + (-2/3)*u0_1*u1_1 + 2/3*u1_3*u1_0 + 4/3*u1_2*u1_1 + (-1/3)*u0_4 + 2/9*u1_5

(-4/3)*u0_1*u1_0 + (-4/3)*u0_0*u1_1 + 2/3*u1_2*u1_0 + 2/3*u1_1^2 + (-2/3)*u0_3 + 1/3*u1_4

### 1.1. The differential system of almost-commuting

* Check the structure of the system to compute $P_m(u)$.
* State the differential integration approach: try and use either [2] or [4] explicitly to solve it.
* State the ansatz approach by linear algebra. This has solution unique due to Wilson's theorem.
* Compare both methods explicitly.

## 2. Looking for specific solutions

Once we have the conditions the differential variables $u = (u_0,\ldots,u_{n-2})$ has to satisfy in order to obtain an $L$ with a non-trivial commutator $P_m(u)$ we can start looking for specific types of solutions. But first of all, we need to see that we may need all the basis of almost-commuting operators:

* If $[L,P] = 0$, then $P$ almost-commutes with $L$.
* Then, if $\ord(P) = m$, then we have $P = \sum_{i=0}^m c_i P_i(u)$ where $c_i\in C$ and $P_i(u)$ are the elements in Wilson's basis.

Hence, in order to look for solutions we need to consider the *complete* set of equations that involve the possible constant coefficients $c_i$ in the previous expression. In order to do so, just remark that $[L,\cdot]$ is a $C$-linear functional, hence:
$$[L,P] = \sum_{i=0}^m c_i[L(u),P_i(u)] = \sum_{i=0}^m \sum_{j=0}^{n-2} c_iH_j^{(n,i)}(u) \partial^j = \sum_{j=0}^{n-2} \left(\sum_{i=0}^m c_iH_j^{(n,i)}(u)\right)\partial^j = \sum_{j=0}^{n-2} H_j^{(P)}(c, u)\partial^j.$$

So for any almost-commuting operator $P$ we obtain a set of $n-1$ equations that can be computed from the equations for Wilson's basis elements. In a matrix-vector multiplication format, it would read as follows:
$$H^{(P)} = \left(H^{(n,0)}|H^{(n,1)}|\cdots|H^{(n,m)}\right)\mathbf{c},$$
where $H^{*} = \left(H_0^{*},\ldots,H_{n-2}^*\right)^T$ and $\mathbf{c} = (c_0,\ldots,c_{m})^T$ is called the flag of constants of $P$.

#### **Some simplifications**

Since we are looking for non-trivial commutators of a given differential operator $L$, we do not need to consider either $L$ or powers of $L$. This can be easily reflected into the previous computation by setting $c_{kn} = 0$, since $P_{kn} = L^k$. 

### 2.1 Looking for specific types of solutions

Assume the differential variables $u = (u_0, \ldots, u_{n-2})$ are of specific shape. This means, each $u_i = U_i(\mathbf{b})$ where $\mathbf{b}$ is a set of constants and $U_i$ is a function describing the shape of $u_i$. 

For example, if we look to polynomial solutions of degree at most $p$, we would have $\mathbf{b} = (b_{i,j}\ :\ i=0,\ldots,n-2; j=0,\ldots,p)$ and then we obtain
$$U_i(\mathbf{b}) = \sum_{j=0}^p b_{i,j}x^j,$$
where $x$ will be a transcendental element with $\partial(x) = 1$.

Plugging these guesses for $u_i = U_i(\mathbf{b})$ into the equations $H^{(P)}$ for a generic flag of constants, we will get then a system of equations on $\mathbf{b}$ and $\mathbf{c}$ that, for each solution, we would have a non-trivial solution to our problem.

#### Example: polynomial solutions fixed $n=3$, $m=4$ and $p=2$

In [2]:
%%time
L, P, H = PolynomialCommutator(3,4,2)

CPU times: user 6.57 s, sys: 851 µs, total: 6.58 s
Wall time: 6.58 s


In [7]:
P

c_4*z_4 + c_3*z_3 + (4/3*b_1_2*x^2*c_4 + 4/3*b_1_1*x*c_4 + 4/3*b_1_0*c_4 + c_2)*z_2 + (b_1_2*x^2*c_3 + 4/3*b_0_2*x^2*c_4 + b_1_1*x*c_3 + 4/3*b_0_1*x*c_4 + 4/3*b_1_2*x*c_4 + b_1_0*c_3 + 4/3*b_0_0*c_4 + 2/3*b_1_1*c_4 + c_1)*z_1 + (2/9*b_1_2^2*x^4*c_4 + 4/9*b_1_1*b_1_2*x^3*c_4 + 2/9*b_1_1^2*x^2*c_4 + 4/9*b_1_0*b_1_2*x^2*c_4 + 2/3*b_1_2*x^2*c_2 + b_0_2*x^2*c_3 + 4/9*b_1_0*b_1_1*x*c_4 + 2/3*b_1_1*x*c_2 + b_0_1*x*c_3 + 2/9*b_1_0^2*c_4 + 4/3*b_0_2*x*c_4 + 2/3*b_1_0*c_2 + b_0_0*c_3 + 2/3*b_0_1*c_4 + 4/9*b_1_2*c_4)*z_0 + c_0

In [4]:
for el in H.basis:
    show(el(**{"c_4": 1, "b_0_1": 0, "b_1_1": 0, "b_0_2": 0, "b_1_2": 0}))

0

0

0

0

0

0

0

0

0

0

In [9]:
L_eval = L.parent()(sum([m*el.wrapped(**{"c_4": 1, "b_0_1": 0, "b_1_1": 0, "b_0_2": 0, "b_1_2": 0}) for m,el in zip(L.polynomial().monomials(), L.coefficients())]))
L_eval

z_3 + b_1_0*z_1 + b_0_0*z_0

In [10]:
P_eval = P.parent()(sum([m*el.wrapped(**{"c_4": 1, "b_0_1": 0, "b_1_1": 0, "b_0_2": 0, "b_1_2": 0}) for m,el in zip(P.polynomial().monomials(), P.coefficients())]))
P_eval

z_4 + c_3*z_3 + (4/3*b_1_0 + c_2)*z_2 + (b_1_0*c_3 + 4/3*b_0_0 + c_1)*z_1 + (2/9*b_1_0^2 + 2/3*b_1_0*c_2 + b_0_0*c_3)*z_0 + c_0

## Differential resultant of two ordinary differential operators

Let $(\mathbb{K}, \partial)$ be a differential field of characterisfic zero and consider two differential operators
$\mathcal{A}, \mathcal{B} \in \mathbb{K}[D]$, where for any $\alpha in \mathbb{K}$:

$$D \alpha = \alpha D + \partial(\alpha).$$

We want to compute the differential resultant of these two operators. This can be done algebraically, since these operators are basically polynomials in the non-commutative variable $D$, using Sylvester resultant.

Currently this structure is not available on the package `dalgebra` and the package `ore_algebra` may not work in all cases to handle these type of operators. 

Anyway, we can reinterpret differential operators as homogeneous differential polynomials in one variable (let call this variable $w(x)$). Then the differential resultant of the two differential operators is the differential resultant of the two differential polynomials eliminating the variable $w(x)$.


#### 1. Setting up the example

In the paper *Algebro-geometric solitonic solutions and Differential Galois Theory*, the authors studied some spectral curves. They considered the Schr\"odinger operator
$$L = -\partial^2  + u,$$
where $u(x)$ is a differential indeterminate that is calle a *potential*. 

Then, they consider some other operators (coming from the KdV hierarchy?):
$$\begin{array}{l}
    \tilde{P_1} = P_1,\\
    \tilde{P_3} = P_3 + c_1P_1,\\
    \tilde{P_5} = P_5 + c_2P_3 + c_1P_1,
\end{array}$$
where the operators $P_1$, $P_3$ and $P_5$ are defined as follows:
$$\begin{array}{l}
    P_1 = \partial,\\
    P_3 = -\partial^3+\frac{3}{2} u \partial +\frac{3}{4}u',\\
    P_5 = \partial^5-\frac{5}{2}u \partial^3-\frac{15}{4}u'\partial^2+\frac{15}{8}u^2\partial-\frac{25}{8}u''\partial -\frac{15}{16}u'''+\frac{15}{8}u u'.
\end{array}$$

The differential resultants they computed were:
$$dres(L - \lambda, \tilde{P}_3 - \mu),\qquad dres(L - \lambda, \tilde{P}_5 - \mu).$$

Hence we have to work on the differential ring: $\mathbb{C}\{u\}[c_i][\lambda, \mu]$ and consider those operators in that ring. We are going to create several variables for each step of the process:

* `B` will represent the ring with the potential: $\mathbb{C}\{u\}$.
* `R` will represent the addition of the constants: $\mathbb{C}\{u\}[c_i]$.
* `S` will represent the addition of the two spectral parameters: $\mathbb{C}\{u\}[c_i][\lambda,\mu]$.
* Finally, `F` will represent the ring with the extra differential variable $w(x)$ that allow us to represent the differential operators as differential polynomials: $\mathbb{C}\{u\}[c_i][\lambda, \mu]\{w\}$.

We are going to rearrange the creation of the ring so the differential computations are performed properly.

For doing so, we are going to compute the ring $\mathbb{C}[c_1,c_2,\lambda,\mu]\{u,w\}$. Then we can set the derivation on the base polynomial ring to be the constant zero (since all the variables there are constant w.r.t. the derivation) and the differential behavior of $u$ and $w$ is preserved properly.

In [6]:
B.<c_1,c_2,l,m> = QQ[]
dB = DifferentialRing(B, lambda p : 0)
F.<u,w> = DifferentialPolynomialRing(dB); F

Ring of operator polynomials in (u, w) over Differential Ring [[Multivariate Polynomial Ring in c_1, c_2, l, m over Rational Field], (0,)]

At this point we can create the differential operators as differential polynomials:

In [7]:
L = -w[2] + u[0]*w[0]

In [8]:
P1 = w[1]
P3 = -w[3] + (3/2)*u[0]*w[1] + (3/4)*u[1]*w[0]
P5 = w[5] - (5/2)*u[0]*w[3] - (15/4)*u[1]*w[2] + (15/8)*u[0]^2*w[1] - (25/8)*u[2]*w[1] - (15/16)*u[3]*w[0] + (15/8)*u[0]*u[1]*w[0]

In [9]:
tP3 = P3 + c_1*P1
tP5 = P5 + c_2*P3 + c_1*P1

In [10]:
op1 = L-l*w[0]; op2 = tP3 - m*w[0]; op3 = tP5 - m*w[0]

#### 2. The first differential resultant

We are going to look now to the differential resultant of `op1` and `op2`, i.e.:
$$dres(L-\lambda, \tilde{P}_3-\mu)$$
eliminating the variable $w$.

For doing so, we need to use the class `DifferentialSystem` indicating the two differential polynomials involved and the variables we will eliminate:

In [11]:
system1 = DifferentialSystem([op1, op2], variables=[w])
system1

System over [Ring of operator polynomials in (u, w) over Differential Ring [[Multivariate Polynomial Ring in c_1, c_2, l, m over Rational Field], (0,)]] with variables [(w_*,)]:
{
	u_0*w_0 - w_2 + (-l)*w_0 == 0
	3/4*u_1*w_0 + 3/2*u_0*w_1 - w_3 + c_1*w_1 + (-m)*w_0 == 0
}

In [13]:
dres_system1 = system1.diff_resultant(alg_res="macaulay")
dres_system1

1/4*u_0^3 + (-1/8)*u_2*u_0 + 1/16*u_1^2 + (c_1 + 3/4*l)*u_0^2 + (-1/4*c_1 - 1/4*l)*u_2 + (c_1^2 + c_1*l)*u_0 - c_1^2*l - 2*c_1*l^2 - l^3 - m^2

At this point, to extrac the coefficients we will need to change the order of the variables appearing in this polynomial:

**WARNING: Maybe this is something we can do automatically... however this comes from the fact that the original structure we wanted to create failed cuurently in `dalgebra`**

#### 3. The second differential resultant

We are going to look now to the differential resultant of `op1` and `op3`, i.e.:
$$dres(L-\lambda, \tilde{P}_5-\mu)$$
eliminating the variable $w$.

For doing so, we need to use the class `DifferentialSystem` indicating the two differential polynomials involved and the variables we will eliminate:

In [14]:
system2 = DifferentialSystem([op1, op3], variables=[w])
system2

System over [Ring of operator polynomials in (u, w) over Differential Ring [[Multivariate Polynomial Ring in c_1, c_2, l, m over Rational Field], (0,)]] with variables [(w_*,)]:
{
	u_0*w_0 - w_2 + (-l)*w_0 == 0
	15/8*u_1*u_0*w_0 + 15/8*u_0^2*w_1 + (-15/16)*u_3*w_0 + (-25/8)*u_2*w_1 + (-15/4)*u_1*w_2 + 3/4*c_2*u_1*w_0 + (-5/2)*u_0*w_3 + 3/2*c_2*u_0*w_1 + w_5 + (-c_2)*w_3 + c_1*w_1 + (-m)*w_0 == 0
}

In [16]:
dres_system2 = system2.diff_resultant(alg_res="macaulay")
dres_system2

9/64*u_0^5 + (-15/64)*u_2*u_0^3 + (3/8*c_2 + 15/64*l)*u_0^4 + 3/128*u_4*u_0^2 + (-3/64)*u_3*u_1*u_0 + 1/16*u_2^2*u_0 + 3/64*u_2*u_1^2 + (-13/32*c_2 - 5/16*l)*u_2*u_0^2 + (1/4*c_2^2 + 7/8*c_2*l + 5/8*l^2 + 3/4*c_1)*u_0^3 + (-1/128)*u_4*u_2 + (1/32*c_2 + 1/32*l)*u_4*u_0 + 1/256*u_3^2 + (-1/32*c_2 - 1/32*l)*u_3*u_1 + (1/32*c_2 + 1/64*l)*u_2^2 + (-1/8*c_2^2 - 3/4*c_2*l - 5/8*l^2 - 5/8*c_1)*u_2*u_0 + (1/16*c_2^2 - 1/4*c_2*l - 5/16*l^2 - 3/8*c_1)*u_1^2 + (3/4*c_2^2*l + 3/4*c_2*l^2 + c_1*c_2 + 1/4*c_1*l)*u_0^2 + (1/16*c_2*l + 1/16*l^2 + 1/16*c_1)*u_4 + (-1/4*c_2^2*l - 1/4*c_2*l^2 - 1/4*c_1*c_2)*u_2 + (c_1*c_2*l + c_1*l^2 + c_1^2)*u_0 - c_2^2*l^3 - 2*c_2*l^4 - l^5 - 2*c_1*c_2*l^2 - 2*c_1*l^3 - c_1^2*l - m^2

Once again, we reorganize the variable to be more readable with respect to the variables $\lambda$ and $\mu$: